In [1]:
import chaospy as cp
import pandas as pd
import numpy as np
import respy as rp

In [2]:
import os
import chaospy as cp

## Create sample

In [144]:
model= "kw_97_extended"

load model specification

In [145]:
_, options = rp.get_example_model(model, with_data=False)

In [146]:
params = pd.read_pickle("./input/respy-se-collection/params_kw_97_extended_respy_se.pkl")
params 

value        se
category               name                                        
delta                  delta                       0.9371  0.005192
wage_white_collar      constant                    8.7419  0.070230
                       exp_school                  0.0655  0.004303
                       exp_white_collar            0.0176  0.007348
                       exp_white_collar_square    -0.0422  0.026438
...                                                   ...       ...
lagged_choice_1_school eight_years_of_schooling   -1.7580       NaN
                       nine_years_of_schooling    -0.2123       NaN
                       ten_years_of_schooling     -0.0231       NaN
                       eleven_years_of_schooling  -0.0097       NaN
maximum_exp            school                     20.0000       NaN

[115 rows x 2 columns]

In [147]:
constraints = rp.get_parameter_constraints(model)
constraints

[{'loc': 'shocks_sdcorr', 'type': 'sdcorr'},
 {'loc': [('initial_exp_school_7', 'probability'),
   ('initial_exp_school_8', 'probability'),
   ('initial_exp_school_9', 'probability'),
   ('initial_exp_school_10', 'probability'),
   ('initial_exp_school_11', 'probability')],
  'type': 'fixed'},
 {'loc': 'maximum_exp', 'type': 'fixed'},
 {'query': "name == 'military_dropout'", 'type': 'equality'},
 {'query': "name == 'common_hs_graduate'", 'type': 'equality'},
 {'query': "name == 'common_co_graduate'", 'type': 'equality'},
 {'loc': 'lagged_choice_1_school', 'type': 'fixed'},
 {'loc': 'lagged_choice_1_home', 'type': 'fixed'}]

In [148]:
constraints.remove({"loc": "shocks_sdcorr", "type": "sdcorr"})

load mean and cov

In [149]:
cov = pd.read_pickle("./input/respy-se-collection/covariance_kw_97_extended_respy.pkl")
cov

category                                        delta wage_white_collar  \
name                                            delta          constant   
category          name                                                    
delta             delta                      0.000027          0.000162   
wage_white_collar constant                   0.000162          0.004932   
                  exp_school                -0.000009         -0.000253   
                  exp_white_collar          -0.000004         -0.000105   
                  exp_white_collar_square    0.000026          0.000541   
...                                               ...               ...   
type_1            at_least_ten_years_school -0.000090         -0.001586   
type_2            up_to_nine_years_school   -0.000078         -0.001109   
                  at_least_ten_years_school -0.000096         -0.001070   
type_3            up_to_nine_years_school    0.000027         -0.001240   
                  at_least_ten_years_school -0.000113         -0.000826   

category                                                                 \
name                                        exp_school exp_white_collar   
category          name                                                    
delta             delta                      -0.000009        -0.000004   
wage_white_collar constant                   -0.000253        -0.000105   
                  exp_school                  0.000019         0.000007   
                  exp_white_collar            0.000007         0.000054   
                  exp_white_collar_square    -0.000026        -0.000175   
...                                                ...              ...   
type_1            at_least_ten_years_school   0.000076        -0.000027   
type_2            up_to_nine_years_school     0.000098        -0.000041   
                  at_least_ten_years_school   0.000056        -0.000030   
type_3            up_to_nine_years_school     0.000091         0.000082   
                  at_least_ten_years_school   0.000069        -0.000067   

category                                                             \
name                                        exp_white_collar_square   
category          name                                                
delta             delta                                    0.000026   
wage_white_collar constant                                 0.000541   
                  exp_school                              -0.000026   
                  exp_white_collar                        -0.000175   
                  exp_white_collar_square                  0.000699   
...                                                             ...   
type_1            at_least_ten_years_school                0.000245   
type_2            up_to_nine_years_school                  0.000354   
                  at_least_ten_years_school                0.000262   
type_3            up_to_nine_years_school                 -0.000055   
                  at_least_ten_years_school                0.000395   

category                                                                  \
name                                        exp_blue_collar exp_military   
category          name                                                     
delta             delta                           -0.000011    -0.000003   
wage_white_collar constant                        -0.000091    -0.000056   
                  exp_school                       0.000007     0.000005   
                  exp_white_collar                 0.000008     0.000012   
                  exp_white_collar_square         -0.000019    -0.000017   
...                                                     ...          ...   
type_1            at_least_ten_years_school        0.000042     0.000054   
type_2            up_to_nine_years_school          0.000076     0.000025   
                  at_least_ten_years_school        0.000039     0.0

In [150]:
mean = pd.read_pickle("./input/respy-se-collection/params_kw_97_extended_respy_se.pkl")
mean = mean.reindex(cov.index)["value"]
mean

category           name                     
delta              delta                        0.9371
wage_white_collar  constant                     8.7419
                   exp_school                   0.0655
                   exp_white_collar             0.0176
                   exp_white_collar_square     -0.0422
                                                 ...  
type_1             at_least_ten_years_school   -0.0034
type_2             up_to_nine_years_school      2.3501
                   at_least_ten_years_school    0.4646
type_3             up_to_nine_years_school      2.0134
                   at_least_ten_years_school   -0.3759
Name: value, Length: 88, dtype: float64

get sampled parameters

In [151]:
np.random.seed(123)

In [152]:
distribution = cp.MvNormal(loc=mean, scale=cov)

In [43]:
sample_array = distribution.sample(size=1000, rule='random' )

In [44]:
list(sample_array.T)

[array([ 9.39770068e-01,  8.67044038e+00,  6.85660306e-02,  3.22444688e-02,
        -1.01805063e-01,  3.41191661e-02,  1.14927597e-02,  4.99863620e-03,
        -3.70032060e-04,  9.47930017e-03, -1.29465844e-01,  1.88601883e-01,
         2.89506223e-01, -4.30367130e-02, -5.91480753e-01, -4.52351320e-01,
        -2.55569052e+03, -1.52620837e+03, -3.33679860e+03,  6.05297498e+02,
         1.64563895e+03, -1.52643717e+03,  8.80455573e+00,  2.18919504e-02,
         1.24437061e-02,  3.64431507e-02, -8.48239943e-02,  1.69358062e-02,
         2.01675364e-03,  1.07632595e-02,  1.28413738e-02,  1.41887748e-01,
         2.12233247e-01,  9.65470252e-02,  3.33436060e-01, -1.83089574e-01,
        -3.43458932e-02, -3.15752337e+03, -1.58898692e+03, -1.37830959e+03,
         8.30169969e+00,  5.70984506e-02,  6.85684421e-02, -2.58224911e-01,
         4.22722240e-02,  1.25352913e-02, -2.54338398e-01,  1.28064063e+03,
         9.62851914e+03, -1.95741065e+04, -8.64210958e+03, -1.43474136e+03,
         3.2

## script_uncertainty_propagation

propagate mean estimates

In [170]:
 _, base_options = rp.get_example_model("kw_94_one", with_data=False)

In [171]:
mean_params = pd.read_pickle("./input/respy-se-collection/params_kw_94_one_se.pkl")
mean_params

value          se
category            name                                           
delta               delta                        0.9500    0.000616
wage_a              constant                     9.2100    0.011858
                    exp_edu                      0.0380    0.001045
                    exp_a                        0.0330    0.000444
                    exp_a_square                -0.0005    0.000012
                    exp_b                        0.0000    0.000738
                    exp_b_square                 0.0000    0.000029
wage_b              constant                     8.4800    0.005998
                    exp_edu                      0.0700    0.000344
                    exp_b                        0.0670    0.000546
                    exp_b_square                -0.0010    0.000016
                    exp_a                        0.0220    0.000330
                    exp_a_square                -0.0005    0.000022
nonpec_edu          constant                     0.0000  244.837595
                    at_least_twelve_exp_edu      0.0000  134.591673
                    not_edu_last_period      -4000.0000  209.141590
nonpec_home         constant                 17750.0000  270.436557
shocks_chol         chol_a                       0.2000    0.001388
                    chol_b_a                     0.0000    0.006947
                    chol_b                       0.2500    0.001334
                    chol_edu_a                   0.0000  136.909459
                    chol_edu_b                   0.0000  110.539536
                    chol_edu                  1500.0000  119.506921
                    chol_home_a                  0.0000  153.723203
                    chol_home_b                  0.0000  103.725464
                    chol_home_edu                0.0000  168.278822
                    chol_home                 1500.0000  138.848240
lagged_choice_1_edu probability                  1.0000         NaN
initial_exp_edu_10  probability                  1.0000         NaN
maximum_exp         edu                         20.0000         NaN

In [172]:
def model_wrapper_kw_94(input_params, base_options, tuition_subsidy):
    simulate = rp.get_simulate_func(input_params, base_options)

    policy_params = input_params.copy()
    policy_params.loc[
        ("nonpec_edu", "at_least_twelve_exp_edu"), "value"
    ] += tuition_subsidy
    policy_df = simulate(policy_params)

    edu = policy_df.groupby("Identifier")["Experience_Edu"].max().mean()

    occ_shares_df = (
        policy_df.groupby(['Identifier','Period'])
        .Choice.value_counts(normalize=True)
        .unstack()[["home", "edu", "a", "b"]]
    )
    # Set 0 NaNs in edu shares to 0.
    occ_shares_df["edu"].fillna(0, inplace=True)

    return edu, occ_shares_df, policy_df


# # policy_edu, base_occ_shares_df, _ = model_wrapper_kw_94(mean_params, base_options, 500)

# simulate = rp.get_simulate_func(mean_params, base_options)

In [122]:
policy_edu, base_occ_shares_df, _ = model_wrapper_kw_94(
    mean_params, base_options, 500
)

In [123]:
base_edu, policy_occ_shares_df, _ = model_wrapper_kw_94(
    mean_params, base_options, 0
)

In [124]:
qoi_mean_params_edu = policy_edu - base_edu

In [125]:
mean_edu_df = pd.DataFrame(
    qoi_mean_params_edu, columns=["change_mean_schooling"], index=[0]
)

In [126]:
mean_edu_df

,change_mean_schooling
0,1.496


In [129]:
base_occ_shares_df

Choice             home  edu   a    b
Identifier Period                    
0          0        NaN  1.0 NaN  NaN
           1        NaN  1.0 NaN  NaN
           2        NaN  0.0 NaN  1.0
           3        NaN  1.0 NaN  NaN
           4        NaN  1.0 NaN  NaN
...                 ...  ...  ..  ...
999        35       NaN  0.0 NaN  1.0
           36       NaN  0.0 NaN  1.0
           37       NaN  0.0 NaN  1.0
           38       NaN  0.0 NaN  1.0
           39       NaN  0.0 NaN  1.0

[40000 rows x 4 columns]

In [130]:
policy_occ_shares_df

Choice             home  edu    a    b
Identifier Period                     
0          0        NaN  0.0  1.0  NaN
           1        NaN  0.0  NaN  1.0
           2        NaN  0.0  NaN  1.0
           3        NaN  1.0  NaN  NaN
           4        NaN  1.0  NaN  NaN
...                 ...  ...  ...  ...
999        35       NaN  0.0  NaN  1.0
           36       NaN  0.0  NaN  1.0
           37       NaN  0.0  NaN  1.0
           38       NaN  0.0  NaN  1.0
           39       NaN  0.0  NaN  1.0

[40000 rows x 4 columns]

Init estimates of parameters and their covariance matrix as nummpy arrays.

In [159]:
params = pd.read_pickle("./input/respy-se-collection/params_kw_94_one_se.pkl")
mean = params["value"].to_numpy()
cov = pd.read_pickle("./input/respy-se-collection/covariance_kw_94_one.pkl").to_numpy()

In [163]:
mean = mean[:27]

Draw the sample of random input parameters.

In [165]:
np.random.seed(123)
distribution = cp.MvNormal(loc=mean, scale=cov)

In [166]:
mc_sample_input_parameters = list()
for _ in range(1000):
    mc_sample_input_parameters.append(distribution.sample())


## 20210314

load model specification(params, options, constraints)

In [3]:
model= "kw_94_one"
_, base_options = rp.get_example_model(model, with_data=False)

In [4]:
base_params = pd.read_pickle("./input/respy-se-collection/params_kw_94_one_se.pkl")

In [5]:
constraints = rp.get_parameter_constraints(model)
constraints.remove({"loc": "shocks_sdcorr", "type": "sdcorr"})
constraints

[{'loc': 'lagged_choice_1_edu', 'type': 'fixed'},
 {'loc': 'initial_exp_edu_10', 'type': 'fixed'},
 {'loc': 'maximum_exp', 'type': 'fixed'}]

Unconditional samples

In [6]:
mean = base_params["value"].to_numpy()[:27]
cov = pd.read_pickle("./input/respy-se-collection/covariance_kw_94_one.pkl").to_numpy()

In [7]:
np.random.seed(123)
distribution = cp.MvNormal(loc=mean, scale=cov)

In [69]:
# x = np.array([distribution.sample() for _ in range(100)])
# x_prime = np.array([distribution.sample() for _ in range(100)])
x = [distribution.sample() for _ in range(100)]
# x = list(map(list, zip(*x)))
x[0]

array([ 9.50135336e-01,  9.22533205e+00,  3.61073086e-02,  3.32339324e-02,
       -5.01444547e-04,  5.57745976e-05,  3.81435023e-06,  8.48242662e+00,
        7.00841635e-02,  6.59934405e-02, -9.68292246e-04,  2.22589812e-02,
       -5.26318463e-04,  1.08265862e+02, -7.21059644e+01, -4.05923848e+03,
        1.76227931e+04,  2.00143728e-01, -9.57706400e-03,  2.49731615e-01,
        4.13158493e+01,  2.36190623e+01,  1.58724108e+03, -1.08619971e+02,
       -1.68382765e+02, -9.21950935e+01,  1.40413976e+03])

conditional_samples

In [40]:
n_draws = 100
n_params = 27
x_mix = np.zeros((n_draws, n_params, n_draws, n_params))

for i in range(n_params):
    for j in range(n_draws):
        x_mix[j, i] = x
        x_mix[j, i, :, i] = x_prime[j, i]

In [83]:
def multi_quantities_of_interest(input_params, *args):
    # We need the baseline options and a grid for the indices.
    # It does not matter which of the three KW94 specifications we use here.
    _, base_options = rp.get_example_model("kw_94_one", with_data=False)
    # Estimated parameters with Choleskies plus 3 fixed respy parameters.
    base_params = pd.read_pickle("./input/respy-se-collection/params_kw_94_one_se.pkl")

    params_idx = pd.Series(data=input_params, index=base_params.index[0:27])
    params_idx_respy = transform_params_kw94_respy(params_idx)

    policy_edu = model_wrapper_kw_94(
        params_idx_respy, base_options, 500.0
    )
    base_edu = model_wrapper_kw_94(
        params_idx_respy, base_options, 0.0
    )
    change_mean_edu = policy_edu - base_edu

    return change_mean_edu

In [84]:
def model_wrapper_kw_94(input_params, base_options, tuition_subsidy):
    simulate = rp.get_simulate_func(input_params, base_options)

    policy_params = input_params.copy()
    policy_params.loc[
        ("nonpec_edu", "at_least_twelve_exp_edu"), "value"
    ] += tuition_subsidy
    policy_df = simulate(policy_params)

    edu = policy_df.groupby("Identifier")["Experience_Edu"].max().mean()

    return edu

In [85]:
def transform_params_kw94_respy(params_idx):
    assert len(params_idx) == 27, "Length of KW94 vector must be 27."
    part_1 = params_idx

    rp_params, _ = rp.get_example_model("kw_94_one", with_data=False)
    part_2 = rp_params.iloc[27:31, 0]

    parts = [part_1, part_2]
    rp_params_series = pd.concat(parts)
    rp_params_df = pd.DataFrame(rp_params_series, columns=["value"])

    return rp_params_df

In [ ]:
mc_change_mean_edu = list()
for i in map(multi_quantities_of_interest, x):
    mc_change_mean_edu.append(i)

transform x to respy format

In [60]:
params_idx_list = []
for param in x:
    params_idx = pd.Series(data=param[0], index=base_params.index[0:27])
    params_idx_list.append(params_idx)


In [61]:
assert len(params_idx_list[1]) == 27, "Length of KW94 vector must be 27."
part_1 = params_idx_list[1]

In [62]:
part_1

category     name                   
delta        delta                      9.211988
wage_a       constant                   9.211988
             exp_edu                    9.211988
             exp_a                      9.211988
             exp_a_square               9.211988
             exp_b                      9.211988
             exp_b_square               9.211988
wage_b       constant                   9.211988
             exp_edu                    9.211988
             exp_b                      9.211988
             exp_b_square               9.211988
             exp_a                      9.211988
             exp_a_square               9.211988
nonpec_edu   constant                   9.211988
             at_least_twelve_exp_edu    9.211988
             not_edu_last_period        9.211988
nonpec_home  constant                   9.211988
shocks_chol  chol_a                     9.211988
             chol_b_a                   9.211988
             chol_b             

In [56]:
rp_params, _ = rp.get_example_model("kw_94_one", with_data=False)
part_2 = rp_params.iloc[27:31, 0]

parts = [part_1, part_2]
rp_params_series = pd.concat(parts)
rp_params_df = pd.DataFrame(rp_params_series, columns=["value"])

In [57]:
rp_params_df

value
category            name                              
delta               delta                     0.949893
wage_a              constant                  0.949893
                    exp_edu                   0.949893
                    exp_a                     0.949893
                    exp_a_square              0.949893
                    exp_b                     0.949893
                    exp_b_square              0.949893
wage_b              constant                  0.949893
                    exp_edu                   0.949893
                    exp_b                     0.949893
                    exp_b_square              0.949893
                    exp_a                     0.949893
                    exp_a_square              0.949893
nonpec_edu          constant                  0.949893
                    at_least_twelve_exp_edu   0.949893
                    not_edu_last_period       0.949893
nonpec_home         constant                  0.949893
shocks_chol         chol_a                    0.949893
                    chol_b_a                  0.949893
                    chol_b                    0.949893
                    chol_edu_a                0.949893
                    chol_edu_b                0.949893
                    chol_edu                  0.949893
                    chol_home_a               0.949893
                    chol_home_b               0.949893
                    chol_home_edu             0.949893
                    chol_home                 0.949893
lagged_choice_1_edu probability               1.000000
initial_exp_edu_10  probability               1.000000
maximum_exp         edu                      20.000000

In [34]:
simulate = rp.get_simulate_func(rp_params_df, base_options)

TypeError: loop of ufunc does not support argument 0 of type float which has no callable log method

Get simulate function

In [11]:
simulate_func = rp.get_simulate_func(x, options)

NameError: name 'options' is not defined

Compute qoi

In [179]:
params = params[["value"]]

In [180]:
# base case
base_params = params.copy()
base_df = simulate_func(base_params)

In [183]:
# policy case
subsidy_params = params.copy()
subsidy_params.loc[("nonpec_edu", "at_least_twelve_exp_edu"),
              "value"] += 500
policy_df = simulate_func(subsidy_params)

In [184]:
# preparations
qois = {}
base_df = base_df.copy()
policy_df = policy_df.copy()
vf_cols = [c for c in base_df.columns if "Value_Function" in c]
base_df["Value_Function_Max"] = base_df[vf_cols].max(axis=1)
policy_df["Value_Function_Max"] = policy_df[vf_cols].max(axis=1)
last_period = base_df.index.get_level_values("Period").max()

In [190]:
# calculate overall effect of tuition subsidy on years of education
base_last_period = base_df.query(f"Period == {last_period}")
policy_last_period = policy_df.query(f"Period == {last_period}")

base_education = base_last_period["Experience_Edu"].mean()
policy_education = policy_last_period["Experience_Edu"].mean()
policy_effect = policy_education - base_education
qois["subsidy_effect_on_years"] = {
    "difference": policy_effect,
    "baseline": base_education,
    "with_subsidy": policy_education,
}

In [192]:
qois

{'subsidy_effect_on_years': {'difference': 1.4959999999999987,
  'baseline': 12.55,
  'with_subsidy': 14.046}}

Quantile-based sensitivity analysis

In [195]:
# range of alpha
dalp = (0.98 - 0.02) / 30
alpha_grid = np.arange(0.02, 0.98 + dalp, dalp)  # len(alpha_grid) = 31

unconditional_quantile_y

load model specification(params, options, constraints)

In [3]:
model= "kw_94_one"
_, base_options = rp.get_example_model(model, with_data=False)

In [9]:
base_params = pd.read_pickle("./input/respy-se-collection/params_kw_94_one_se.pkl")
# if "lower" not in params.columns:
#     params["lower"] = -np.inf
#     params["upper"] = np.inf
# params 

#  params_idx_respy = transform_params_kw94_respy(params_idx)

In [10]:
# transfer sampled params into respy format
params_idx = pd.Series(data=x, index=base_params.index[0:27])

NameError: name 'x' is not defined

In [ ]:
assert len(params_idx) == 27, "Length of KW94 vector must be 27."
part_1 = params_idx

rp_params, _ = rp.get_example_model("kw_94_one", with_data=False)
part_2 = rp_params.iloc[27:31, 0]

parts = [part_1, part_2]
rp_params_series = pd.concat(parts)
rp_params_df = pd.DataFrame(rp_params_series, columns=["value"])

In [176]:
constraints = rp.get_parameter_constraints(model)
constraints.remove({"loc": "shocks_sdcorr", "type": "sdcorr"})
constraints

[{'loc': 'lagged_choice_1_edu', 'type': 'fixed'},
 {'loc': 'initial_exp_edu_10', 'type': 'fixed'},
 {'loc': 'maximum_exp', 'type': 'fixed'}]